In [90]:
import numpy as np
import random
import pandas as pd
import os
import re
import seaborn as sns


In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
data_energy = pd.read_csv('/Users/pptrv/IronHack/Week_3-Project/Data/energy_dataset.csv', encoding ='latin1')
data_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal                        35046 non-null  float64
 10

First creating a new dataset with the columns for date and types of generation:

In [129]:
data_energy = data_energy[['time', 'generation biomass', 'generation fossil brown coal/lignite','generation fossil coal-derived gas','generation fossil gas','generation fossil hard coal','generation fossil oil', 'generation fossil oil shale', 'generation fossil peat', 'generation geothermal', 'generation hydro pumped storage aggregated', 'generation hydro pumped storage consumption', 'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation marine','generation nuclear', 'generation other', 'generation other renewable','generation solar','generation waste', 'generation wind offshore', 'generation wind onshore']]

The analisis will be focused on the repercusions of my three favourite renewable energy generators; wind, solar and hydro. Therefore, all other means of energy generation will be merged as a new column : "other_generations". 

In [130]:
data_energy['other forms of generations'] = data_energy['generation biomass'] + data_energy['generation fossil brown coal/lignite'] + data_energy['generation fossil coal-derived gas'] + data_energy['generation fossil gas'] + data_energy['generation fossil hard coal'] + data_energy['generation fossil oil'] + data_energy['generation fossil oil shale'] + data_energy['generation fossil peat'] + data_energy['generation geothermal'] + data_energy['generation marine'] + data_energy['generation nuclear'] + data_energy['generation other'] + data_energy['generation other renewable'] + data_energy['generation waste']


Furthermore, the three main types of renewable energy we are analysing are found separated in other columns such as wind offshore or onshore, so therefore we will unite them as a unique form of generation for each type 

Hydro:

In [131]:
data_energy['hydro generation'] = data_energy['generation hydro pumped storage aggregated'] + data_energy['generation hydro pumped storage consumption'] + data_energy['generation hydro run-of-river and poundage'] + data_energy['generation hydro water reservoir']

Wind:

In [132]:
data_energy['wind generation'] = data_energy['generation wind offshore'] + data_energy['generation wind onshore']

Further simplification of the dataset keeping only those columns neeeded:

In [134]:
data_energy = data_energy[['time', 'wind generation', 'hydro generation', 'generation solar', 'other forms of generations']]  

In [135]:
data_energy = data_energy.rename(columns={'time': 'Dates & Hours'})

In [136]:
data_energy = data_energy.rename(columns={'generation solar': 'solar generation'})

In [137]:
data_energy_clean = data_energy

In [138]:
data_energy_clean.sample(10)

,Dates & Hours,wind generation,hydro generation,solar generation,other forms of generations
25577,2017-12-01 17:00:00+01:00,8369.0,NaN,1225.0,23138.0
8420,2015-12-17 20:00:00+01:00,5487.0,NaN,35.0,23139.0
30870,2018-07-10 07:00:00+02:00,7676.0,NaN,41.0,14907.0
27174,2018-02-06 06:00:00+01:00,1526.0,NaN,122.0,14900.0
12481,2016-06-04 02:00:00+02:00,7394.0,NaN,340.0,12576.0
30930,2018-07-12 19:00:00+02:00,7468.0,NaN,84.0,16761.0
26626,2018-01-14 10:00:00+01:00,3127.0,NaN,661.0,18905.0
5816,2015-08-31 09:00:00+02:00,2290.0,NaN,811.0,25071.0
17408,2016-12-26 08:00:00+01:00,2028.0,NaN,61.0,17798.0
23785,2017-09-18 02:00:00+02:00,3381.0,NaN,478.0,15360.0


In [139]:
data_energy_clean.sort_values('Dates & Hours')

,Dates & Hours,wind generation,hydro generation,solar generation,other forms of generations
0,2015-01-01 00:00:00+01:00,6378.0,NaN,49.0,18011.0
1,2015-01-01 01:00:00+01:00,5890.0,NaN,50.0,18291.0
2,2015-01-01 02:00:00+01:00,5461.0,NaN,50.0,17777.0
3,2015-01-01 03:00:00+01:00,5238.0,NaN,50.0,16704.0
4,2015-01-01 04:00:00+01:00,4935.0,NaN,42.0,16144.0
...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,3113.0,NaN,85.0,17245.0
35060,2018-12-31 20:00:00+01:00,3288.0,NaN,33.0,16788.0
35061,2018-12-31 21:00:00+01:00,3503.0,NaN,31.0,16424.0
35062,2018-12-31 22:00:00+01:00,3586.0,NaN,31.0,15827.0
